# 2.3.3 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot
* Chain-Of-Thought
* Best result: A2

In [1]:
from utils.parser import Parser
from utils.openai import OpenAI
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
samantha = OpenAI(instructions)
parser = Parser()

In [2]:
pdf_path = "../data/raw/bpläne/3_xplanung/BP_872A_AUFST_TT.pdf"
pdf_prompts = parser.pdf2prompts(pdf_path)
print(len(pdf_prompts))

66


In [4]:
# A.1) Ein Thread – Analyse und Extraktion in einem Schritt
# Problem: Kontext ist sehr groß, sodass GPT den Überblick verliert und Informationen verliert. 
msg1 = samantha.request([
    *pdf_prompts,
    {
        "type": "text",
        "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>)'
        # Optional: Zusätzlich Dach, Garage, Stelltung und Stellplatz
    }
])

print(msg1)

### Art der baulichen Nutzung:
- Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.
- Zulässig sind:
  - Wohngebäude
  - Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe
  - Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke
- Nicht zulässig sind:
  - Betriebe des Beherbergungsgewerbes
  - sonstige nicht störende Gewerbebetriebe
  - Anlagen für Verwaltungen
  - Gartenbaubetriebe
  - Tankstellen
- Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### Maß der baulichen Nutzung:
- Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.
- In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anlagen jeweils bis zu folgender GRZ überschritten werden:
  - WA1: GRZ maximal 0,65
  - WA2: GRZ maximal 0,80
  -

In [3]:
# A.2) Ein Thread – Relevante Seiten extrahieren (Vorverarbeitung) – (Bestes Ergebnis)
# Wenn Dokument zu groß wird: z.B. chunk_size=10 pages, overlap=2 pages.
# chunk_size = 10, overlap = 2
# n = math.ceil(len(pdf_prompts)/chunk_size)
# chunks = list(map(lambda i: pdf_prompts[slice(max(i*chunk_size-overlap,0),i*chunk_size+chunk_size)], range(n)))
msg = samantha.request([
        *pdf_prompts,
        {
            "type": "text",
            "text": 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zu folgenden Themen beinhalten: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise und überbaubare Grundstücksfläche. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'
        }
    ])

print(msg)

```json
{
  "Seiten": [6, 7, 8, 9, 28, 29, 30, 31, 32, 33]
}
```


In [11]:
# A.2) Ein Thread – Inhalt extrahieren basierend auf relevante Seiten – Bestest Ergebnis
msg2_content = [6, 7, 8, 28, 29, 30, 31, 32, 33]
relevant_pages = list(map(lambda index: pdf_prompts[index-1], msg2_content))
print(len(relevant_pages))

msg3 = samantha.request([
    *relevant_pages,
    {
        "type": "text",
        "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise und überbaubare Grundstücksfläche.'
    },
])

print(msg3)

9
Hier sind die extrahierten Informationen zu den Themen "Art der baulichen Nutzung", "Maß der baulichen Nutzung" und "Bauweise und überbaubare Grundstücksfläche" aus dem Bebauungsplan:

### Art der baulichen Nutzung (§ 4)
1. **Allgemeines Wohngebiet (WA)**
   - Zulässig:
     - Wohngebäude
     - Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe zur Versorgung des Gebiets
     - Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke
   - Nicht zulässig:
     - Betriebe des Beherbergungsgewerbes
     - Sonstige nicht störende Gewerbebetriebe
     - Anlagen für Verwaltungen
     - Gartenbaubetriebe
     - Tankstellen
   - Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### Maß der baulichen Nutzung (§ 5)
1. **Festsetzung des Maßes der baulichen Nutzung**
   - In den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.
2. **Zulässige Grundflächenzahl (GRZ) in den allgemeinen Wohngebieten**
   - WA1: GRZ maxima

In [12]:
# B) Mehrere Threads
# Idee: Jede Seite einzeln verarbeiten, um Context-Window klein zu halten.
# Problem: Informationen sind auf mehrere Seiten verteilt => fehlender Kontext => unvollständige Resultate.
async def run(pdf_prompt):
    msg = samantha.request([
        {
            "type": "text",
            "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Steht keine relevante Information zur Verfügung gebe "-" aus.'
        },
        pdf_prompt
    ])
    return msg.content

prompt_chain = list(map(lambda pdf_prompt: run(pdf_prompt), pdf_prompts))
results = await asyncio.gather(*prompt_chain)
for result in results:
    print(result)